In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, fixed, interact, interactive, interact_manual, Label, Layout, Textarea

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 20ex !important; font-weight: bold;}
.widget-checkbox {margin-left: -60px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))

class ConfigForm(object):
    # Define widgets
    caption = widgets.HTML(value='<h3>Configure Your Action</h3>')
    button = widgets.Button(description='Submit')
    action = widgets.Dropdown(options=['Import', 'Delete', 'Update', 'Search'], value='Import')
    delete = widgets.Text(value='')
    update = widgets.Text(value='')
    properties = Textarea(value='', placeholder="A dict, e.g. {'title': 'New Title', 'description': 'New description'}")
    placeholder = "Enter property-value pairs to search on separate lines: e.g.\n\n'property1': 'value1',\n'property2': 'value2'"
    query = widgets.Textarea(value='', placeholder=placeholder)
    regex = widgets.Checkbox(value=False, description='Use regex')
    limit = widgets.IntText(value=10)
    show_properties = Textarea(value='', placeholder="List of properties to show on separate lines.")
    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    form_items = [
        Box([Label(value='Action:'), action], layout=flex),
        Box([Label(value='`_id` to Delete:'), delete], layout=flex),
        Box([Label(value='`_id` to Update:'), update], layout=flex),
        Box([Label(value='Properties to Update:'), properties], layout=flex),
        Box([Label(value='Query:'), query], layout=flex),
        Box([regex], layout=flex),
        Box([Label(value='Number of Results to Show:'), limit], layout=flex),
        Box([Label(value='Properties to Show:'), show_properties], ayout=flex),
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self.action = ConfigForm.form_items[0]
        self.delete = ConfigForm.form_items[1]
        self.update = ConfigForm.form_items[2]
        self.properties = ConfigForm.form_items[3]
        self.query = ConfigForm.form_items[4]
        self.regex = ConfigForm.form_items[5]
        self.limit = ConfigForm.form_items[6]
        self.show_properties = ConfigForm.form_items[7]

        # Modify widget visibility when the form is changed
        def change_visibility(change):
            box = self.form
            if change['new'] == 'Delete':
                box.children[1].add_class('visible').remove_class('hidden')
                for item in box.children[2:]:
                    item.remove_class('visible').add_class('hidden')
            elif change['new'] == 'Update':
                for item in box.children[4:]:
                    item.remove_class('visible').add_class('hidden')
                box.children[1].remove_class('visible').add_class('hidden')
                box.children[2].add_class('visible').remove_class('hidden')
                box.children[3].add_class('visible').remove_class('hidden')
            elif change['new'] == 'Search':
                for item in box.children[2:4]:
                    item.remove_class('visible').add_class('hidden')
                for item in box.children[4:]:
                    item.add_class('visible').remove_class('hidden')

        def reshape_query_props(temp_query, temp_show_properties):   
            # Convert the query string to a dict of properties
            query_props = {}
            for item in temp_query.split('\n'):
                prop, val = item.split(':')
                prop = prop.strip().strip('"').strip("'")
                val = val.strip().strip('"').strip("'")
                query_props[prop] = val
            # Convert the properties to show to a list
            show_props = temp_show_properties.split('\n')
            return query_props, show_props
                    
        def handle_submit(values):
            # Save the form values in a dict
            self.values = {'action': ConfigForm.action.value}
            if ConfigForm.action.value == 'Delete':
                self.values['delete_id'] = ConfigForm.delete.value 
            if ConfigForm.action.value == 'Update':
                self.values['update_id'] = ConfigForm.update.value
                self.values['properties'] = ConfigForm.properties.value
            if ConfigForm.action.value == 'Search':
                self.values['regex'] = ConfigForm.regex.value
                self.values['limit'] = ConfigForm.limit.value
                temp_query, temp_show_properties = reshape_query_props(ConfigForm.query.value, ConfigForm.show_properties.value)
                self.values['query'] = temp_query
                self.values['show_properties'] = temp_show_properties
            print('Configuration saved. Values will be available is the cells below.')

        # Initialise widgets in the container Box
        self.form = Box([self.action, self.delete, self.update, self.properties, 
                         self.query, self.regex, self.limit, self.show_properties],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form
        for item in box.children[1:]:
            item.add_class('hidden')
        action_field = box.children[0].children[1]

        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.button)
        action_field.observe(change_visibility)
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)